# Face Mask Detection Using Convolutional Neural Network(CNN)

<img src="cnn.jpg"/>

<p style="text-align:justify;">Convolutional Neural Network is a type of deep learning algorithm commonly used in image processing tasks such as image classification, object detection, segmentation, and more.
The main idea behind CNNs is to learn spatial hierarchies of features from the input image. It consists of multiple layers, each layer performing a specific operation such as convolution, pooling, and activation.</p>

### Importing Libraries 

In [2]:
import os
import cv2
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.utils import np_utils
from sklearn.metrics import confusion_matrix
from tensorflow.keras.models import load_model
from sklearn.model_selection import train_test_split

## Part 1 - Data Preparation and Preprocessing

In [3]:
data_path = 'Dataset'
img_class = ['without_mask', 'with_mask']
labels = {'without_mask': 0, 'with_mask': 1}
img_size = 100
numOfCategory = 2

### Declaring variables to store <b>Data</b> and <b>Target</b>

In [ ]:
data = []
target = []

### Accessing the dataset to load the data

In [ ]:
for eachClass in img_class:
    folder_path = os.path.join(data_path, eachClass)
    img_names = os.listdir(folder_path)
    
    for img_name in img_names:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
          
        try:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            resized = cv2.resize(gray, (img_size, img_size))
            
            data.append(resized)
            target.append(labels[eachClass])
            
        except Exception as e:
            
            print("Exception: ", e)

In [ ]:
print(len(data))
print(len(target))

### Convert Data & Target into Array + Normalization

In [ ]:
readyData = np.array(data)/255.0
readyData = readyData.reshape(len(readyData), img_size, img_size, 1)
readyTarget = np.array(target)
readyTarget = np_utils.to_categorical(readyTarget)

### Export Data & Target 

In [ ]:
np.save('data', readyData)
np.save('target', readyTargets)

## Part 2 - Model Training 
### Load the preprocessed data

In [ ]:
trainData = np.load('data.npy')
trainTarget = np.load('target.npy')

### Splitting data into training set and testing set
#### Training data: The training dataset is the data used to train a machine learning model.
#### Testing data: The testing dataset is used to evaluate the performance of a trained machine learning model.
#### Validation data: The validation dataset is used to tune the hyperparameters of a machine learning model. 


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(trainData, trainTarget, test_size=0.2, random_state=42)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

### Define the CNN model

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

model = keras.Sequential(
    [
        layers.Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(img_size, img_size, 1)),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(64, kernel_size=(3, 3), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(numOfCategory, activation='softmax')
    ]
)

### Compile the model


In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the model

In [ ]:
history = model.fit(x_train, y_train, epochs=5, validation_data=(x_val, y_val))

### Evaluate the model on the test set


In [ ]:
test_loss, test_acc = model.evaluate(x_test, y_test)
print('Test accuracy:', test_acc)

### Plot the training and validation accuracy 

<p style="text-align:justify;"><b>To check for overfitting by comparing the training and validation loss and accuracy</b> <br>If the training loss and accuracy continue to improve while the validation loss and accuracy start to plateau or even decrease, it may indicate that the model is overfitting the training data.</p>
<img src="accuracyGraph.jpg" style="width:500px;">

In [ ]:
plt.plot(history.history['accuracy'], label='Training accuracy')
plt.plot(history.history['val_accuracy'], label='Validation accuracy')
plt.legend()
plt.show()

### Construct Confusion Matrix for Evaluation


<img src="confusionMatrix.jpg" style="width:500px;">

In [ ]:
y_pred = model.predict(x_test)

y_pred = np.argmax(y_pred, axis=1)
y_test = np.argmax(y_test, axis=1)
cm = confusion_matrix(y_test, y_pred)
tn, fp, fn, tp = confusion_matrix(y_test, y_pred).ravel()

In [ ]:
plt.figure(figsize=(8,6))
sns.heatmap(cm, annot=True, cmap="Blues", fmt="d", cbar=False)
plt.title("Confusion Matrix")
plt.xlabel("Predicted Labels")
plt.ylabel("Actual Labels")
plt.show()

In [ ]:
accuracy = (tp + tn) / (tp + tn + fp + fn)
precision = tp / (tp + fp)
recall = tp / (tp + fn)
f1_score = 2 * (precision * recall) / (precision + recall)
print("Accuracy: ", np.format_float_positional(accuracy, precision=4))
print("Precision: ", np.format_float_positional(precision, precision=4))
print("Recall: ", np.format_float_positional(recall, precision=4))
print("F1_Score: ", np.format_float_positional(f1_score, precision=4))

### Export Model 

In [ ]:
model.save('model_CNN.h5')

## Part 3 - Model Deployment & Application 

### Load the saved model


In [4]:
model = load_model('model_CNN.h5')

### Define the output of prediction classes


In [5]:
classes = ['Masked', 'No Mask']

### Deploy using Real-Time Video

In [10]:
# Testing using real-time video
import pygame
import time
from tensorflow.keras.models import load_model

# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

# Initialize Pygame mixer for sound
pygame.mixer.init()

# Load the alarm sound file
sound = pygame.mixer.Sound("warn.mp3")
# define the gap of alarm in seconds
time_gap = 2
last_alarm = time.time()
# Initialize video capture object
cap = cv2.VideoCapture(0)

# Set font for displaying accuracy
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2

while True:
    # Capture frame-by-frame
    ret, frame = cap.read()

    # Convert the frame to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Loop over all detected faces
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = gray[y:y+h, x:x+w]

        # Resize the face ROI to match the input size of the model
        face_roi_resized = cv2.resize(face_roi, (100, 100))

        # Normalize the face ROI to have pixel values between 0 and 1
        face_roi_norm = face_roi_resized / 255.0

        # Reshape the face ROI to match the input shape of the model
        face_roi_norm_reshaped = np.reshape(face_roi_norm, (1, 100, 100, 1))

        # Make a prediction on the face ROI using the loaded model
        prediction = model.predict(face_roi_norm_reshaped)

        # Get the predicted class label
        predicted_class = np.argmax(prediction)

        # Get the predicted class name
        if predicted_class == 1:
            predicted_class_name = 'Mask'
            color = [0, 255, 0]
        else:
            predicted_class_name = 'No Mask'
            color = [0, 0, 255]
            
        # Get the predicted class probability
        predicted_prob = prediction[0][predicted_class]   
        
        # Draw a rectangle around the face
        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            
        # Put the predicted class and accuracy on the rectangleq
        cv2.putText(frame, predicted_class_name + ' ' + str(round(predicted_prob * 100, 2)) + '%', (x, y-10), font, font_scale, color, thickness)

        # If no mask is detected and alarm is not already on, activate the alarm
        if predicted_class == 0 and time.time() - last_alarm > time_gap:
            sound.play()
            last_alarm = time.time()
            
    # Display the resulting frame
    cv2.imshow('Mask Detection', frame)

    # Wait for 'q' key to be pressed to exit
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

1/1 [==============================] - 0s 28ms/step


1/1 [==============================] - 0s 29ms/step


1/1 [==============================] - 0s 26ms/step


### Deploy using Image
<br>
<div style="display:flex; align-content: center;">
<img src="noMaskOutput.jpg" style="width:350px; display:inline-block; ">
<img src="maskOutput.jpg" style="width:350px; display:inline-block;">
</div>

In [ ]:
# Testing using image
from tensorflow.keras.models import load_model

inputImg = cv2.imread('BlackpinkNoMask.jpg') #BlackpinkNoMask.jpg
# Load the face cascade classifier
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
gray = cv2.cvtColor(inputImg, cv2.COLOR_BGR2GRAY)
# Detect faces in the frame
faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

# Loop over all detected faces
for (x, y, w, h) in faces:
    # Extract the face ROI
    face_roi = gray[y:y+h, x:x+w]

    # Resize the face ROI to match the input size of the model
    face_roi_resized = cv2.resize(face_roi, (100, 100))

    # Normalize the face ROI to have pixel values between 0 and 1
    face_roi_norm = face_roi_resized / 255.0

    # Reshape the face ROI to match the input shape of the model
    face_roi_norm_reshaped = np.reshape(face_roi_norm, (1, 100, 100, 1))

    # Make a prediction on the face ROI using the loaded model
    prediction = model.predict(face_roi_norm_reshaped)

    # Get the predicted class label
    predicted_class = np.argmax(prediction)

    # Get the predicted class name
    if predicted_class == 1:
        predicted_class_name = 'Mask'
        color = [0, 255, 0]
    else:
        predicted_class_name = 'No Mask'
        color = [0, 0, 255]
            
    # Get the predicted class probability
    predicted_prob = prediction[0][predicted_class]   
        
    # Draw a rectangle around the face
    cv2.rectangle(inputImg, (x, y), (x+w, y+h), color, 2)
            
    # Put the predicted class and accuracy on the rectangleq
    cv2.putText(inputImg, predicted_class_name + ' ' + str(round(predicted_prob * 100, 2)) + '%', (x, y-10), font, font_scale, color, thickness)

# Display the resulting frame
cv2.imshow('Mask Detection', inputImg)

cv2.waitKey(0)
cv2.destroyAllWindows()

# Release the video capture object
cap.release()

# Destroy all windows
cv2.destroyAllWindows()

### Deployment with User Interface - Upload Image

In [ ]:
import tkinter as tk
from tkinter import filedialog
from PIL import ImageTk, Image
import cv2
import numpy as np
from tensorflow.keras.models import load_model

model = load_model('model_CNN.h5')

root = tk.Tk()
root.title("Face Mask Detection")

canvas = tk.Canvas(root, width=1000, height=700)
canvas.pack()

# Store a reference to the PhotoImage object
photo_img = None

# Set font for displaying accuracy
font = cv2.FONT_HERSHEY_SIMPLEX
font_scale = 1
thickness = 2


def upload_image():
    file_path = filedialog.askopenfilename()
    img = cv2.imread(file_path)
    # Load the face cascade classifier
    face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Detect faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    # Loop over all detected faces
    for (x, y, w, h) in faces:
        # Extract the face ROI
        face_roi = gray[y:y+h, x:x+w]

        # Resize the face ROI to match the input size of the model
        face_roi_resized = cv2.resize(face_roi, (100, 100))

        # Normalize the face ROI to have pixel values between 0 and 1
        face_roi_norm = face_roi_resized / 255.0

        # Reshape the face ROI to match the input shape of the model
        face_roi_norm_reshaped = np.reshape(face_roi_norm, (1, 100, 100, 1))

        # Make a prediction on the face ROI using the loaded model
        prediction = model.predict(face_roi_norm_reshaped)

        # Get the predicted class label
        predicted_class = np.argmax(prediction)

        # Get the predicted class name
        if predicted_class == 1:
            predicted_class_name = 'Mask'
            color = [0, 255, 0]
        else:
            predicted_class_name = 'No Mask'
            color = [0, 0, 255]
            
        # Get the predicted class probability
        predicted_prob = prediction[0][predicted_class]   
        
        # Draw a rectangle around the face
        cv2.rectangle(img, (x, y), (x+w, y+h), color, 2)
            
        # Put the predicted class and accuracy on the rectangle
        cv2.putText(img, predicted_class_name + ' ' + str(round(predicted_prob * 100, 2)) + '%', (x, y-10), font, font_scale, color, thickness)

 # Convert the BGR image to RGB
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    global photo_img
   
    img = Image.fromarray(img)
    photo_img = ImageTk.PhotoImage(img)
    canvas.create_image(0, 0, anchor=tk.NW, image=photo_img)


btn_upload = tk.Button(root, text='Upload Image', command=upload_image)
btn_upload.pack()

root.mainloop()
